In [1]:
import os, json
import re
import pandas as pd
import numpy as np
import glob
import csv
import warnings
from pandas import isnull

# from google.colab import drive
# drive.mount('/content/drive')

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

pattern = r'(_d[1-8][ab])'
mapped_seq_df = pd.DataFrame()
list_of_sequences = []

# PLEASE CHANGE THE PATH ACCORDINGLY
your_path = '/Users/surya/Documents/UoM_Dissertation/'

# /Users/surya/Documents/UoM_Dissertation/artifacts/data/raw/Interaction Data

raw_path = your_path + 'artifacts/data/raw/'
preprocess_path =  your_path + 'artifacts/data/preprocessing/'
prediction_path = your_path + 'artifacts/data/prediction/'
json_path = raw_path + 'Interaction Data/'
json_files = os.listdir(json_path)

unwanted_columns = [ 'screen', 'client', 'episodeCount',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'scroll', 'mouse', 'delta', 'key', 'change', 'select'
       'link', 'class', 'img', 'type', 'inheritedId', 'name', 'textValue', 'text', 'textContent']

necessary_columns = ['event', 'timestampms','node', 'user','id', 'dom', 'flag', 'task', 'merged_event']

total_columns = ['_id', 'event', 'screen', 'client', 'episodeCount', 'timestampms',
       'sessionstartms', 'timezoneOffset', 'sd', 'sid', 'url', 'urlFull',
       'mouse', 'node', 'scroll', 'delta', 'key', 'select', 'change', 'user',
       'type', 'textContent', 'text', 'link', 'inheritedId', 'img', 'class',
       'id', 'textValue', 'name', 'dom', 'flag', 'task', 'merged_event']

id_values = ['toDashboard1', 'toDashboard2', 'toDashboard3', 'toDashboard4', 'toDashboard5',
             'toDashboard6', 'toDashboard7', 'toDashboard8', 'toDashboard9', 'toDashboard10',
             'toDashboard11', 'toDashboard12', 'toDashboard13', 'toDashboard14', 'toDashboard15',
             'toDashboard16']

id_value_until_practice_tutorial = id_values[0]

surya_df = pd.DataFrame()

/Users/surya/Documents/UoM_Dissertation/artifacts/data/raw/Interaction Data/


### function definitions


In [2]:
### function definitions

def reading_json_files(json_data):
  json_data = re.sub(r'ObjectId\((.+?)\)', r'\1', json_data)
  tempdf = pd.read_json(json_data, lines=True)
  return tempdf


def basic_cleaning(tempdf):
  tempdf['user'] = i
  tempdf.drop(['mobileTouch','evID'], axis=1, inplace = True, errors='ignore')
  return tempdf

def splitting_json_column_values(df):
      # Split the key-value pairs in the node column
      df['node'] = df['node'].apply(lambda x: {} if pd.isnull(x) else x)  # Convert NaN values to empty dictionaries
      # Extract the keys from the node dictionary
      keys = set()
      for node_dict in df['node']:
          keys.update(node_dict.keys())
      # Create new columns for each key
      for key in keys:
          df[key] = [node_dict.get(key) if isinstance(node_dict, dict) else None for node_dict in df['node']]

def updating_dom_element(df):
  count = df[df['dom'].notna() & df['dom'].str.contains('\)')].shape[0]
  # Strip the substring before ')' for non-NA values
  df.loc[df['dom'].notna() & df['dom'].str.contains('\)'), 'dom'] = df['dom'].str.split('\)').str[-1]
  df.reset_index(drop=True, inplace=True)

def removing_rows_of_practice_tutorial(df):
  # filling NA values
  df['id'].fillna(-555, inplace=True)
  # ignoring the indices until the practise tutorial
  last_index = df[df['id'] == id_value_until_practice_tutorial].index[-1]
  # Step 2: Remove all rows before the last index
  df = df.loc[last_index:]
  # Reset the index if desired
  df = df.reset_index(drop=True)

def removing_mid_values_between_toDashboardN_ids(df):
  # Remove mid rows between 'toDashboardN' first and last occurrences
  first_occurrences = {}
  last_occurrences = {}
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      first_occurrences[id_value] = df[df['id'] == id_value].index[0]
      last_occurrences[id_value] = df[df['id'] == id_value].index[-1]
  # Step 2: Remove the rows between the first and last occurrence for each value
  for value, first_index in first_occurrences.items():
      last_index = last_occurrences[value]
      df = df.drop(index=range(first_index , last_index))

def dropping_mid_rows_between_toDashboardN_ids(df):
  dropped_rows_count = {id_value: 0 for id_value in id_values}
  for id_value in id_values:
      # Get the indices of the first and last occurrence of the id value
      first_occurrence = df.index[df['id'] == id_value].min()
      last_occurrence = df.index[df['id'] == id_value].max()
      # Determine the indices of the middle rows between the first and last occurrence
      middle_rows_indices = list(range(first_occurrence , last_occurrence))
      dropped_rows_count[id_value] = len(middle_rows_indices)
      df.drop(middle_rows_indices, inplace=True)

  # Reset the index of the dataframe
  df.reset_index(drop=True, inplace=True)

def flagging_toDashboard_rows(df):
  df['flag'] = False
  # Step 2: Set the flag to True for rows matching the 'toDashboard' values
  for value in range(1, 17):
      id_value = f'toDashboard{value}'
      df.loc[df['id'] == id_value, 'flag'] = True
      df.reset_index(drop= True, inplace=True)

def merging_events(df):
    df['merged_event'] = df['event']
    # Replace 'mousewheel' with 'scroll'
    df['merged_event'] = df['merged_event'].replace('mousewheel', 'scroll')
    # Merge rows based on the initial conditions
    # df.loc[(df['merged_event'] == 'mouseover') & (df['merged_event'].shift(1) == 'mouseout'), 'merged_event'] = 'hover'
    df['merged_event'] = df['merged_event'].str.replace('mouseover', 'hover').str.replace('mouseout', 'hover')

    df['merged_event'] = df['merged_event'].str.replace('mouseup', 'click').str.replace('mousedown', 'click')
    # df.loc[(df['merged_event'] == 'mouseup') & (df['merged_event'].shift(1) == 'mousedown'), 'merged_event'] = 'click'
    df['merged_event'] = df['merged_event'].str.replace('keyup', 'press').str.replace('keydown', 'press')
    df['merged_event'] = df['merged_event'].str.replace('mobileTouchStart', 'click').str.replace('mobileTouchEnd', 'click')

    # df.loc[(df['merged_event'] == 'keyup') & (df['merged_event'].shift(1) == 'keydown'), 'merged_event'] = 'press'
    df.reset_index(drop=True, inplace=True)

def merging_multiple_events(df):
    df['multi_merged_event'] = df['merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['merged_event'].shift(1) == 'hover') & (df['merged_event'] == 'hover'), 'multi_merged_event'] = 'multihover'
    df.loc[(df['merged_event'].shift(1) == 'click') & (df['merged_event'] == 'click'), 'multi_merged_event'] = 'multiclick'
    df.loc[(df['merged_event'].shift(1) == 'press') & (df['merged_event'] == 'press'), 'multi_merged_event'] = 'multipress'
    df.loc[(df['merged_event'].shift(1) == 'scroll') & (df['merged_event'] == 'scroll'), 'multi_merged_event'] = 'multiscroll'
    df.reset_index(drop=True, inplace=True)

def merging_events_master(df):
    df['event_master'] = df['multi_merged_event']
    # Merge consecutive rows based on the additional conditions
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'multihover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'mouseout') & (df['multi_merged_event'] == 'hover'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'hover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.loc[(df['multi_merged_event'].shift(1) == 'multihover') & (df['multi_merged_event'] == 'mouseout'), 'event_master'] = 'multihover'
    df.reset_index(drop=True, inplace=True)

def dropping_multi_hoverings(df):
  df['m_event'] = df['event_master'].ne(df['event_master'].shift()).cumsum()
  df = df[df['event_master'] != 'multihover']  # Remove 'mhover' rows
  df = df.drop_duplicates(subset='m_event', keep='first').drop(columns='m_event')


def initialising_new_column_task(df):
  df['task'] = df['id'].str.extract(pattern)
  df.loc[df['flag'], 'task'] = True
  df.reset_index(drop=True, inplace=True)

def getting_unique_task_values(df_name):
      temp_df = df_name  # Access the DataFrame dynamically using globals()
      # Step 1: Find unique non-NaN values in the 'task' column
      unique_values = temp_df['task'].unique()
      unique_task = next((x for x in unique_values if isinstance(x, str)), None)
      # Step 2: Check if any unique value is a string
      string_pattern = None
      return unique_task

def ignoring_na_dom_values(df):
  df = df[(df['dom'].notna()) | (df['flag'] == True)]
  df.reset_index(drop=True, inplace=True)

def getting_dataframes_task_wise_index(df):
  indices = df.index[df['flag']]
  for i in range(len(indices)-1):
      start = indices[i]
      end = indices[i+1]
      df_name = f"df_db{i+1}"
      globals()[df_name] = df.loc[start:end-1].copy()
  # Create the last DataFrame df_db16 from the last True value to the last row
  start = indices[-1]
  end = df.index[-1]
  df_db16 = df.loc[start:end-1].copy()
  task_wise_split_list = []
  # Step 3: Print the separate DataFrames
  for i in range(1, 16):
      df_name = f"df_db{i}"
      task_wise_split_list.append(globals()[df_name])
  task_wise_split_list.append(df_db16)
  return task_wise_split_list

def dropping_na_values_from_itemset_columns(df_name):
  df_name.loc[df_name['dom'] == '', 'dom'] = np.nan
  df_name.dropna(subset=['dom'], inplace=True)

def dropping_consecutive_rows(df, old_col, new_col):
  consecutive_duplicates = (df['final_event'] == df['final_event'].shift(1))
  # Drop duplicates, keeping only the first occurrence of each consecutive series
  df = df[~consecutive_duplicates]

def dropping_single_transformed_events(df):
  rows_to_remove = df[df['merged_event'] != df['event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)
  df = df.reset_index(drop=True)

def dropping_multi_transformed_events(df):
  rows_to_remove = df[df['multi_merged_event'] != df['merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def dropping_events_master(df):
  rows_to_remove = df[df['event_master'] != df['multi_merged_event']].index -1
  # Step 1: Filter rows to remove to avoid KeyError
  valid_rows_to_remove = [idx for idx in rows_to_remove if idx in df.index]
  # Step 2: Remove the rows from the DataFrame
  df.drop(valid_rows_to_remove, inplace=True)

def defining_sequence(series_list):
  # Concatenate all the series into a single series
  concatenated_series = pd.concat(series_list, ignore_index=True)
  # Append -2 to the end of the final series
  concatenated_series = concatenated_series.append(pd.Series([-2]))
  # removing rows with substring of nan values or index True values from 'Task' column
  sequence = concatenated_series[~concatenated_series.apply(lambda x: str(x).endswith(('Tru_e')))]
  return sequence

def replace_unique_strings_with_numbers(big_string):
    # Split the big string into words and remove duplicates
    words = big_string.split()
    unique_strings = list(set(words))
    # Remove '-1' and '-2' from the list of unique strings
    unique_strings = [s for s in unique_strings if s not in ['-1', '-2']]
    # Create a mapping dictionary between original string values and numbers
    num_mapping = {}
    next_number = 1  # Start assigning numbers from 1000
    for unique_string in unique_strings:
        num_mapping[unique_string] = str(next_number)
        next_number += 1
    # Replace the unique string values in the big string with their corresponding numbers
    for i in range(len(words)):
        if words[i] in num_mapping:
            words[i] = num_mapping[words[i]]
    # Join the words back into a new string
    new_string = ' '.join(words)
    # Create a DataFrame to store the mapping between the original and new string values
    mapping_df = pd.DataFrame(list(num_mapping.items()), columns=['Original_String', 'New_Number'])
    return new_string, mapping_df

def split_string_on_negative_two(input_string):
    # Split the input_string whenever "-2" is encountered
    output_string = re.sub(r'-2\s', '-2\n', input_string)
    return output_string

In [3]:
def defining_itemset(df_name, val):
  df_name['task_val'] =  val
  dropping_na_values_from_itemset_columns(df_name)
  dropping_single_transformed_events(df_name)
  merging_multiple_events(df_name)
  dropping_multi_transformed_events(df_name)
  merging_events_master(df_name)
  dropping_events_master(df_name)

  df_name['itemset'] = df_name['event_master'].astype(str) + "_" + df_name['type'].astype(str)
  # df_name['itemset'] = df_name['merged_event'].astype(str) + "_" + df_name['id'].astype(str) + df_name['dom'].astype(str)
  return df_name['itemset']

def deep_cleaning(d_t_list):
  itemset_column = []
  user_sequence = []
  abc_list = []
  for inner_df in d_t_list:
    it = defining_itemset(inner_df, getting_unique_task_values(inner_df))
    itemset_column.append(it.append(pd.Series([-1])))
    abc_list.append(inner_df)
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  # Step 2: Convert the concatenated Series to a single line string
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  # user_sequence.append(defining_sequence(itemset_column))
  return single_line, abc_list


def major_cleaning(d_list):
  d_t_list = []
  one_concatened_dt_list = []
  master_line = ''
  for df in d_list:
      splitting_json_column_values(df)
      updating_dom_element(df)
      removing_rows_of_practice_tutorial(df)
      dropping_mid_rows_between_toDashboardN_ids(df)

      flagging_toDashboard_rows(df)
      initialising_new_column_task(df)
      getting_unique_task_values(df)
      merging_events(df)
      ignoring_na_dom_values(df)
      d_t_list = getting_dataframes_task_wise_index(df)
      # reforming the dataframe
      line, abc_list = deep_cleaning(d_t_list)
      abc_df = pd.concat(abc_list, ignore_index=True)
      one_concatened_dt_list.append(abc_df)
      master_line = master_line + line
  surya_df = pd.concat(one_concatened_dt_list, ignore_index=True)
  # list_of_sequences.append(d_t_list)
  return master_line, surya_df


In [4]:
def combining_similar_types(one):
  one['up_type'] = one['type']

  type_text = ['B', 'STRONG', 'I', 'P', 'GRAMMARLY-MIRROR', 'GRAMMARLY-EXTENSION', 'CENTER' , 'A']
  type_list = ['LI', 'UL']
  type_heading = ['H2', 'H3', 'H4', 'H5', 'HR']
  type_table = ['TD', 'TH']
  type_button = ['COM-1PASSWORD-BUTTON']
  type_graphic = ['rect', 'tspan', 'path', 'IMG']
  type_basic = ['HTML', 'BODY']
  type_select = ['SELECT', 'OPTION']


  # Use .loc to update 'up_type' column where 'type' column values match the values_to_update
  one.loc[one['type'].isin(type_text), 'up_type'] = 'TEXT'
  one.loc[one['type'].isin(type_heading), 'up_type'] = 'HEADING'
  one.loc[one['type'].isin(type_table), 'up_type'] = 'TABLE'
  one.loc[one['type'].isin(type_button), 'up_type'] = 'BUTTON'
  one.loc[one['type'].isin(type_graphic), 'up_type'] = 'GRAPHIC'
  one.loc[one['type'].isin(type_basic), 'up_type'] = 'BASIC'
  one.loc[one['type'].isin(type_select), 'up_type'] = 'SELECTION'
  one.loc[one['type'].isin(type_list), 'up_type'] = 'LISTING'

  return one

def find_consecutive_pairs(df):
    # Create a new column 'consecutive_pair' to store consecutive pairs
    df['consecutive_master_pair'] = df['event_master'] + ' -> ' + df['event_master'].shift(-1)
    df['consecutive_itemset_pair'] = df['itemset'] + ' -> ' + df['itemset'].shift(-1)
    # Drop the last row as it will have a NaN value for the 'shift(-1)' operation
    df.drop(df.index[-1], inplace=True)
    # Print the consecutive pairs of values in 'event' column
    merged_consecutive_pairs = df['consecutive_master_pair'].unique()

def remove_consecutive_duplicates(df, column_name, event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    new_df = df[mask].copy()
    while True:
        mask = (new_df[column_name] != event) | (new_df[column_name].shift() != event)
        temp_df = new_df[mask]
        if temp_df.equals(new_df):
            break
        new_df = temp_df.copy()
    return new_df

def replace_consecutive_events(df, column_name, old_event, new_event):
    mask = (df[column_name] == old_event) & (df[column_name].shift() == old_event)
    df.loc[mask, column_name] = new_event
    return df

def merge_click_sequences_recursive(df, column_name, event, new_event):
    mask = (df[column_name] != event) | (df[column_name].shift() != event)
    df = df[mask]
    if any(df[column_name] == event):
        df = replace_consecutive_events(df, column_name, event, new_event)
        return merge_click_sequences_recursive(df, column_name, event, new_event)
    else:
        return df

def common_values_across_db(series_list):
  # Initialize a set with the values from the first series
  common_values = set(series_list[0])
  # Iterate through the remaining series and update the common_values set
  for series in series_list[1:]:
      common_values.intersection_update(series)
  # Convert the common_values set back to a list if needed
  common_values_list = list(common_values)

  # Print the common values
  print("Common values:", common_values_list)

def user_mapping_with_timestamp(full):
  user_mapping = full[['user','sessionstartms']]
  list_time = user_mapping.sessionstartms.unique()
  data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
  user_time = pd.DataFrame(data)
  user_time.to_csv(preprocess_path+'usertimee.csv',index=False)


In [5]:
d_list = [None] * len(json_files)
# d_list = [None] * 10
i = 1

for file in json_files:
    with open(json_path + file, 'r') as fi:
        json_data = fi.read()
    tempdf = basic_cleaning(reading_json_files(json_data))
    d_list[i-1] = tempdf
    i = i+1
    # if i>10:
    #   break


In [6]:
master_df = pd.concat(d_list, ignore_index=True)
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1114761 entries, 0 to 1114760
Data columns (total 20 columns):
 #   Column          Non-Null Count    Dtype         
---  ------          --------------    -----         
 0   _id             1090484 non-null  object        
 1   event           1114761 non-null  object        
 2   screen          50 non-null       object        
 3   client          50 non-null       object        
 4   episodeCount    1114761 non-null  int64         
 5   timestampms     1114761 non-null  datetime64[ns]
 6   sessionstartms  1114761 non-null  int64         
 7   timezoneOffset  1114761 non-null  int64         
 8   sd              1114761 non-null  int64         
 9   sid             464350 non-null   object        
 10  url             1114761 non-null  object        
 11  urlFull         1114761 non-null  object        
 12  mouse           833011 non-null   object        
 13  node            1082820 non-null  object        
 14  scroll          30

In [7]:
master_df.url.value_counts()

http://ec2-18-170-45-231.eu-west-2.compute.amazonaws.com/study/shinyapp/    1088876
http://127.0.0.1:6713/                                                        25885
Name: url, dtype: int64

In [8]:
master_df.timezoneOffset.value_counts()

-60     555639
 0      211008
-180    198177
 360     89616
-120     60321
Name: timezoneOffset, dtype: int64

In [9]:
master_df.describe()

,episodeCount,sessionstartms,timezoneOffset,sd,delta,user
count,1.114761e+06,1.114761e+06,1.114761e+06,1114761.0,14266.000000,1.114761e+06
mean,1.665614e+00,1.635194e+12,-3.945865e+01,777.0,-0.685942,2.569533e+01
std,1.500583e+00,1.505011e+09,1.311301e+02,0.0,8.199605,1.401337e+01
min,1.000000e+00,1.632465e+12,-1.800000e+02,777.0,-109.650000,1.000000e+00
25%,1.000000e+00,1.634077e+12,-6.000000e+01,777.0,-3.000000,1.400000e+01
50%,1.000000e+00,1.634983e+12,-6.000000e+01,777.0,-1.000000,2.500000e+01
75%,2.000000e+00,1.636725e+12,0.000000e+00,777.0,1.250000,3.700000e+01
max,7.000000e+00,1.638791e+12,3.600000e+02,777.0,181.875000,5.000000e+01


In [10]:
master_df.event.value_counts()

mouseover           385267
mouseout            379054
keydown              75974
keyup                65983
keypress             55334
mouseup              36196
change               34512
mousedown            31156
scroll               30847
mousewheel           14390
select                1981
dblclick              1158
mobileTouchEnd         826
mobileTouchStart       763
windowqueryfocus       540
windowqueryblur        504
contextmenu            180
load                    50
copy                    24
cut                     20
paste                    2
Name: event, dtype: int64

In [11]:
file_value, full = major_cleaning(d_list)
user_mapping_with_timestamp(full)
full.head()

,_id,event,screen,client,episodeCount,timestampms,sessionstartms,timezoneOffset,sd,sid,url,urlFull,mouse,node,scroll,delta,key,change,select,user,textContent,name,class,textValue,img,id,inheritedId,dom,type,flag,task,merged_event,task_val,multi_merged_event,event_master,itemset,text,link
0,6164104f4622cf0027242b69,mouseover,NaN,NaN,1,2021-10-11 10:22:07.300,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 2445, 'coordY': 1253, 'offsetX': 24...","{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,NaN,NaN,NaN,NaN,1,\n,None,[content-wrapper],None,None,tab-2855-6,True,/DIV[1]/DIV[1]/DIV[1],DIV,False,NaN,hover,_d7b,hover,hover,hover_DIV,NaN,NaN
1,616410514622cf0027242b6a,change,NaN,NaN,1,2021-10-11 10:22:09.362,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,NaN,"{'dom': 'id(""dr_d7b"")/DIV[1]/INPUT[1]', 'id': ...",NaN,NaN,NaN,"{'type': 'text', 'value': '2020-01-27'}",NaN,1,null,None,[form-control],2020-01-27,None,dr_d7b,True,/DIV[1]/INPUT[1],INPUT,False,_d7b,change,_d7b,change,change,change_INPUT,NaN,NaN
2,616410994622cf0027242c82,mouseout,NaN,NaN,1,2021-10-11 10:23:21.387,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 1312, 'coordY': 925, 'offsetX': 126...","{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,NaN,NaN,NaN,NaN,1,\n,None,[row],None,None,tab-2855-6,True,/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,DIV,False,NaN,hover,_d7b,multihover,multihover,multihover_DIV,NaN,NaN
3,616410994622cf0027242c84,mousewheel,NaN,NaN,1,2021-10-11 10:23:21.483,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,NaN,"{'dom': 'id(""tab-2855-6"")/DIV[1]/DIV[1]/DIV[1]...",NaN,-18.691667,NaN,NaN,NaN,1,\n,None,[row],None,None,tab-2855-6,True,/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,DIV,False,NaN,scroll,_d7b,scroll,scroll,scroll_DIV,NaN,NaN
4,616410b24622cf0027242d3c,mouseout,NaN,NaN,1,2021-10-11 10:23:46.017,1633946848251,-180,777,NaN,http://ec2-18-170-45-231.eu-west-2.compute.ama...,http://ec2-18-170-45-231.eu-west-2.compute.ama...,"{'coordX': 2937, 'coordY': 164, 'offsetX': 65,...","{'dom': 'id(""id1_d7b"")/DIV[1]/LABEL[1]/SPAN[1]...",NaN,NaN,NaN,NaN,NaN,1,Random 2,None,[],None,None,id1_d7b,True,/DIV[1]/LABEL[1]/SPAN[1],SPAN,False,_d7b,hover,_d7b,multihover,multihover,multihover_SPAN,NaN,NaN


In [12]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17294 entries, 0 to 17293
Data columns (total 38 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   _id                 16855 non-null  object        
 1   event               17294 non-null  object        
 2   screen              0 non-null      object        
 3   client              0 non-null      object        
 4   episodeCount        17294 non-null  int64         
 5   timestampms         17294 non-null  datetime64[ns]
 6   sessionstartms      17294 non-null  int64         
 7   timezoneOffset      17294 non-null  int64         
 8   sd                  17294 non-null  int64         
 9   sid                 7576 non-null   object        
 10  url                 17294 non-null  object        
 11  urlFull             17294 non-null  object        
 12  mouse               12474 non-null  object        
 13  node                17294 non-null  object    

In [13]:
full.type.value_counts()

DIV                     5429
SPAN                    2630
rect                    2001
TD                      1835
INPUT                   1404
SECTION                  799
H3                       780
I                        367
STRONG                   304
TH                       294
BODY                     272
HTML                     243
BUTTON                   223
SELECT                   167
KBD                      146
LABEL                    137
P                         71
A                         52
HR                        25
H5                        20
LI                        19
H2                        15
GRAMMARLY-MIRROR          15
path                      15
OPTION                    12
H4                         5
CENTER                     5
IMG                        3
tspan                      3
GRAMMARLY-EXTENSION        1
UL                         1
COM-1PASSWORD-BUTTON       1
Name: type, dtype: int64

In [14]:
final_string, mapping_dataframe = replace_unique_strings_with_numbers(file_value)
output_string = split_string_on_negative_two(final_string)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path +  'one.txt', 'w') as file:
    file.write(output_string)
# mapping_dataframe

CSV file

In [15]:
find_consecutive_pairs(full)


In [16]:
one = full[['event_master','type', 'itemset', 'id','class','dom','user','task_val', 'consecutive_master_pair', 'consecutive_itemset_pair']]
new_df = remove_consecutive_duplicates(combining_similar_types(one), 'event_master', 'multihover')
new_df.dropna(subset=['task_val'], inplace=True)
find_consecutive_pairs(new_df)


/var/folders/pd/5c5685_16pg8rhq7k0050lt80000gn/T/ipykernel_41801/3118859913.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  one['up_type'] = one['type']


In [17]:
new_df

,event_master,type,itemset,id,class,dom,user,task_val,consecutive_master_pair,consecutive_itemset_pair,up_type
0,hover,DIV,hover_DIV,tab-2855-6,[content-wrapper],/DIV[1]/DIV[1]/DIV[1],1,_d7b,hover -> change,hover_DIV -> change_INPUT,DIV
1,change,INPUT,change_INPUT,dr_d7b,[form-control],/DIV[1]/INPUT[1],1,_d7b,change -> multihover,change_INPUT -> multihover_DIV,INPUT
2,multihover,DIV,multihover_DIV,tab-2855-6,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,1,_d7b,multihover -> scroll,multihover_DIV -> scroll_DIV,DIV
3,scroll,DIV,scroll_DIV,tab-2855-6,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[5]/DIV[1]...,1,_d7b,scroll -> multihover,scroll_DIV -> multihover_SPAN,DIV
4,multihover,SPAN,multihover_SPAN,id1_d7b,[],/DIV[1]/LABEL[1]/SPAN[1],1,_d7b,multihover -> click,multihover_SPAN -> click_INPUT,SPAN
...,...,...,...,...,...,...,...,...,...,...,...
17287,multihover,DIV,multihover_DIV,tab-9738-21,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[2],50,_d4b,multihover -> scroll,multihover_DIV -> scroll_SECTION,DIV
17288,scroll,SECTION,scroll_SECTION,tab-9738-21,[content],/DIV[1]/DIV[1]/DIV[1]/SECTION[1],50,_d4b,scroll -> multihover,scroll_SECTION -> multihover_DIV,SECTION
17289,multihover,DIV,multihover_DIV,line7_d4b,"[user-select-none, svg-container]",/DIV[1]/DIV[1],50,_d4b,multihover -> scroll,multihover_DIV -> scroll_DIV,DIV
17290,scroll,DIV,scroll_DIV,line7_d4b,"[user-select-none, svg-container]",/DIV[1]/DIV[1],50,_d4b,scroll -> multihover,scroll_DIV -> multihover_DIV,DIV


In [18]:
sorted_df = new_df.sort_values(by=['user','task_val'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

# Create an empty list to store the DataFrames
dataframe_sorted_db_list = []
dataframe_sorted_user_db_list = []

db_user_list = []
for user in unique_user_val:
    user_task_df = sorted_df[sorted_df['user'] == user].copy()
    dataframe_sorted_user_db_list.append(user_task_df)

mapped_dashboard_unique_itemsets = []
user_dashboard_wise_seq = []


In [19]:
full_seq = ''
for df in dataframe_sorted_user_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  unique_its = []
  for task in unique_task_val:
    task_df = df[df['task_val'] == task].copy()
    task_df['its'] = task_df['event_master'].astype(str) + '__' + task_df['up_type'].astype(str)
    unique_its.append(task_df['its'])
    itemset_column.append(task_df['its'].append(pd.Series([-1])))
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"
  full_seq = full_seq + single_line


In [20]:
final, mapping_ut_dataframe = replace_unique_strings_with_numbers(full_seq)
output_full_string = split_string_on_negative_two(final)
# Create and write the output to a text file named 'output.txt'
with open(preprocess_path+'full_m_2.txt', 'w') as file:
    file.write(output_full_string)
mapping_ut_dataframe

,Original_String,New_Number
0,select__SELECTION,1
1,multihover__GRAPHIC,2
2,multipress__BUTTON,3
3,hover__HEADING,4
4,select__GRAPHIC,5
...,...,...
108,select__TEXT,109
109,dblclick__TABLE,110
110,multipress__INPUT,111
111,scroll__KBD,112


In [21]:
mapping_ut_dataframe.to_csv(preprocess_path+"Mapped_values.csv")

In [22]:
# surya

sorted_df = new_df.sort_values(by=['task_val','user'])
unique_task_val = sorted_df['task_val'].unique()
unique_user_val = sorted_df['user'].unique()

dataframe_sorted_db_list = []
dashboard_wise_seq = []

# db_user_list = []
for task in unique_task_val:
    user_task_df = sorted_df[sorted_df['task_val'] == task].copy()

    dataframe_sorted_db_list.append(user_task_df)

for df in dataframe_sorted_db_list:
  itemset_column = []
  concatenated_series = pd.Series()
  # unique_its = []
  for user_no in unique_user_val:
    user_df = df[df['user'] == user_no].copy()
    user_df['its'] = user_df['event_master'].astype(str) + '__' + user_df['up_type'].astype(str) + '__' + user_df['user'].astype(str)
    itemset_column.append(user_df['its'].append(pd.Series([-1])))
  # itemset_column.append(unique_its.append())
  concatenated_series = pd.concat(itemset_column, ignore_index=True)
  single_line = " ".join(concatenated_series.astype(str))
  single_line = single_line + " -2\n"

  s1, m1 = replace_unique_strings_with_numbers(single_line)
  mapped_dashboard_unique_itemsets.append(m1)
  dashboard_wise_seq.append(s1)
  # common_values_across_db(unique_its)


In [23]:
dataframe_sorted_db_list[0]

,event_master,type,itemset,id,class,dom,user,task_val,consecutive_master_pair,consecutive_itemset_pair,up_type
49,click,STRONG,click_STRONG,shiny-modal,[],/DIV[1]/DIV[1]/DIV[2]/P[1]/STRONG[1],1,_d1a,click -> select,click_STRONG -> select_STRONG,TEXT
50,select,STRONG,select_STRONG,shiny-modal,[],/DIV[1]/DIV[1]/DIV[2]/P[1]/STRONG[1],1,_d1a,select -> dblclick,select_STRONG -> dblclick_STRONG,TEXT
51,dblclick,STRONG,dblclick_STRONG,shiny-modal,[],/DIV[1]/DIV[1]/DIV[2]/P[1]/STRONG[1],1,_d1a,dblclick -> multihover,dblclick_STRONG -> multihover_DIV,TEXT
52,multihover,DIV,multihover_DIV,shiny-modal,[modal-body],/DIV[1]/DIV[1]/DIV[2],1,_d1a,multihover -> hover,multihover_DIV -> hover_DIV,DIV
315,multiscroll,SECTION,multiscroll_SECTION,tab-7250-18,[content],/DIV[1]/DIV[1]/DIV[1]/SECTION[1],2,_d1a,multiscroll -> multihover,multiscroll_SECTION -> multihover_rect,SECTION
...,...,...,...,...,...,...,...,...,...,...,...
16871,multihover,DIV,multihover_DIV,shiny-modal,[modal-body],/DIV[1]/DIV[1]/DIV[2],49,_d1a,multihover -> change,multihover_DIV -> change_INPUT,DIV
17108,click,DIV,click_DIV,tab-9738-10,[row],/DIV[1]/DIV[1]/DIV[1]/SECTION[1]/DIV[3]/DIV[1]...,50,_d1a,click -> multihover,click_DIV -> multihover_DIV,DIV
17109,multihover,DIV,multihover_DIV,shiny-modal,[modal-body],/DIV[1]/DIV[1]/DIV[2],50,_d1a,multihover -> click,multihover_DIV -> click_SPAN,DIV
17110,click,SPAN,click_SPAN,shiny-modal,[irs],/DIV[1]/DIV[1]/DIV[2]/DIV[3]/SPAN[1]/SPAN[1],50,_d1a,click -> multihover,click_SPAN -> multihover_DIV,SPAN


In [24]:
mapped_dashboard_unique_itemsets[0]

,Original_String,New_Number
0,multihover__TEXT__43,1
1,click__TEXT__17,2
2,multihover__SPAN__3,3
3,click__DIV__32,4
4,multihover__DIV__40,5
...,...,...
446,scroll__SPAN__39,447
447,multihover__DIV__44,448
448,multiscroll__DIV__26,449
449,multiclick__BASIC__38,450


In [25]:
dashboard_wise_seq[0]

'89 22 352 171 -1 365 395 398 86 110 395 398 443 59 338 72 395 296 395 296 410 59 410 385 138 296 410 398 262 398 262 398 202 296 410 167 410 -1 220 324 283 132 283 289 283 289 98 355 324 76 324 283 289 270 289 348 76 348 15 345 283 23 19 348 283 289 283 289 283 289 270 132 384 324 55 3 83 3 440 348 -1 360 258 303 228 48 412 48 69 48 69 94 228 46 412 46 412 48 69 94 -1 331 45 331 45 331 45 35 61 35 61 248 331 45 331 45 371 331 45 139 371 177 215 -1 149 80 232 224 149 41 232 244 149 80 232 80 -1 321 65 -1 265 315 343 432 203 231 203 230 414 230 414 432 -1 287 73 239 81 287 73 239 280 112 111 112 111 112 81 -1 341 16 341 247 263 105 128 105 128 441 263 16 341 16 341 441 451 441 172 247 263 247 -1 435 210 435 210 435 273 24 28 93 28 -1 411 319 272 226 429 113 -1 347 317 161 96 347 60 356 96 326 96 -1 330 400 350 58 206 58 206 58 350 82 350 104 350 104 350 217 70 400 21 401 423 104 350 67 90 104 445 104 206 431 222 67 222 423 165 67 90 104 213 207 424 58 -1 369 442 433 292 200 292 200 442 

In [26]:
merged_string = ''.join(dashboard_wise_seq)
op_merged = split_string_on_negative_two(merged_string)
with open(preprocess_path+'op_merged.txt', 'w') as fi:
      fi.write(op_merged)

# dashboard_wise_seq[0]

# dataframe_sorted_db_list

In [27]:
# # Function to write strings into new files
# def write_strings_to_files(string_list):
#     for i, string in enumerate(string_list):
#         print(len(string))
#         file_name = f"ab_{i+1}.txt"  # Assuming filenames like file_0.txt, file_1.txt, etc.
#         with open(file_name, 'w') as file:
#             file.write(string)

# write_strings_to_files(dashboard_wise_seq)

In [28]:
mapped_dashboard_unique_itemsets

[           Original_String New_Number
 0     multihover__TEXT__43          1
 1          click__TEXT__17          2
 2      multihover__SPAN__3          3
 3           click__DIV__32          4
 4      multihover__DIV__40          5
 ..                     ...        ...
 446       scroll__SPAN__39        447
 447    multihover__DIV__44        448
 448   multiscroll__DIV__26        449
 449  multiclick__BASIC__38        450
 450    scroll__SECTION__10        451
 
 [451 rows x 2 columns],
             Original_String New_Number
 0            click__DIV__33          1
 1       scroll__HEADING__49          2
 2           click__TEXT__17          3
 3    multiscroll__TABLE__39          4
 4          click__TABLE__15          5
 ..                      ...        ...
 513         click__TEXT__49        514
 514     multihover__DIV__44        515
 515   multiclick__BASIC__38        516
 516    multiscroll__DIV__39        517
 517     scroll__SECTION__10        518
 
 [518 rows x 2 columns]

In [29]:
new_df.to_csv(preprocess_path+"two.csv", index=False)


In [30]:
find_consecutive_pairs(new_df)

In [31]:
new_df.consecutive_master_pair.value_counts()

multihover -> click          2435
click -> multihover          2153
scroll -> multihover         1474
multihover -> scroll         1412
multihover -> multiclick      716
                             ... 
press -> multiscroll            2
multiclick -> contextmenu       1
multihover -> contextmenu       1
multihover -> dblclick          1
dblclick -> multipress          1
Name: consecutive_master_pair, Length: 78, dtype: int64

In [32]:
new_df.consecutive_itemset_pair.value_counts()

multihover_SPAN -> click_SPAN           708
multihover_DIV -> scroll_DIV            427
click_SPAN -> multihover_SPAN           401
click_SPAN -> multihover_DIV            384
change_INPUT -> multihover_DIV          312
                                       ... 
press_BODY -> hover_SPAN                  1
hover_SPAN -> press_BODY                  1
select_SECTION -> multiclick_SECTION      1
multiclick_SECTION -> select_SECTION      1
scroll_STRONG -> multihover_SPAN          1
Name: consecutive_itemset_pair, Length: 1756, dtype: int64

In [33]:
full.event_master.value_counts()

multihover     6251
click          2995
hover          2172
scroll         2166
multiclick     1085
multiscroll     962
change          853
select          303
press           257
multipress      187
dblclick         58
contextmenu       4
Name: event_master, dtype: int64

In [34]:
# masterq_df.info()

In [35]:
full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17293 entries, 0 to 17292
Data columns (total 40 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   _id                       16854 non-null  object        
 1   event                     17293 non-null  object        
 2   screen                    0 non-null      object        
 3   client                    0 non-null      object        
 4   episodeCount              17293 non-null  int64         
 5   timestampms               17293 non-null  datetime64[ns]
 6   sessionstartms            17293 non-null  int64         
 7   timezoneOffset            17293 non-null  int64         
 8   sd                        17293 non-null  int64         
 9   sid                       7576 non-null   object        
 10  url                       17293 non-null  object        
 11  urlFull                   17293 non-null  object        
 12  mouse             

In [36]:
user_mapping = full[['user','sessionstartms']]
list_time = user_mapping.sessionstartms.unique()
data = { 'User': np.arange(1, len(list_time) + 1), 'Timestamp': list_time }
user_time = pd.DataFrame(data)
user_time.to_csv(preprocess_path+'usertime.csv',index=False)

In [37]:
# cc

# def get_unique_values_except_minus_one(input_lines):
#     unique_values = set()

#     for line in input_lines:
#         values = line.split()
#         for value in values:
#             try:
#                 int_value = int(value)
#                 if int_value != -1:
#                     unique_values.add(int_value)
#             except ValueError:
#                 pass

#     return unique_values

# input_text = """
# 277 -1 692 -1 811 -1
# 277 -1 692 -1 417 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# """

# input_lines = input_text.strip().split('\n')
# unique_values = get_unique_values_except_minus_one(input_lines)

# print("Unique values except -1:", unique_values)


In [38]:
# from collections import Counter
# import re

# input_text = """
# 277 -1 692 -1 811 -1
# 277 -1 692 -1 417 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 550 -1
# 289 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 727 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 289 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 883 -1 738 -1 222 -1
# 179 -1 879 -1 551 -1 623 -1 423 -1 776 -1 738 -1 222 -1
# """


# # Extract numbers from the input_text using regular expression
# numbers = re.findall(r'\d+', input_text)

# # Convert the numbers to integers and count their occurrences
# number_counts = Counter(map(int, numbers))

# data_count = {'New_Number': [], 'Count': []}
# # Print the count of each unique value
# for value, count in number_counts.items():
#     # print(f"Value: {value}, Count: {count}")

#     data_count['New_Number'].append(value)
#     data_count['Count'].append(count)

# df_data_count = pd.DataFrame(data_count)
# df_data_count



In [39]:
# mapping_ut_dataframe['New_Number'] = mapping_ut_dataframe['New_Number'].astype('int64')
# # mapping_ut_dataframe.info()

In [40]:
# df_data_count.info()

In [41]:
# merged_df_final = pd.merge(mapping_ut_dataframe, df_data_count, on='New_Number')
# merged_df_final['Last_Three_Characters'] = merged_df_final['Original_String'].str[-3:]
# merged_df_final.sort_values(by='Last_Three_Characters',inplace=True)
# merged_df_final

In [42]:
# sum_count = merged_df_final['Count'].sum()
# print("Sum of Count column:", sum_count)

In [43]:
# merged_df_final['Last_Three_Characters']

In [44]:
# cumulative_sum_df = merged_df_final.groupby('Last_Three_Characters')['Count'].cumsum()
# merged_df_final['Cumulative_Count'] = cumulative_sum_df

# merged_df_final


In [45]:
# highest_cumulative_counts = merged_df_final.groupby('Last_Three_Characters')['Cumulative_Count'].max()
# print(highest_cumulative_counts.sort_values(ascending=False))